In [1]:
import pandas as pd
from numpy import loadtxt
import numpy as np
#from hmmlearn.hmm import GaussianHMM
import math
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import sounddevice as sd

In [2]:
data = pd.read_csv("F.txt", delimiter = "\t", names = ["V1", "V2", "V3", "V4"])

In [3]:
voices = ["V1", "V2", "V3", "V4"]
for v in voices:
    prev = 0
    data[v+"-Diff"] = 0 #creating difference representations needed for clustering (according to Dirst)
    for index, row in data.iterrows():
        data[v+"-Diff"][index] = row[v] - prev
        prev = row[v]

In [4]:
data.tail()

,V1,V2,V3,V4,V1-Diff,V2-Diff,V3-Diff,V4-Diff
3819,0,0,49,0,0,0,0,0
3820,0,0,51,0,0,0,2,0
3821,0,0,51,0,0,0,0,0
3822,0,0,54,0,0,0,3,0
3823,0,0,54,0,0,0,0,0


In [5]:
voices_diff = ["V1-Diff", "V2-Diff", "V3-Diff", "V4-Diff"]
voices_l = []

for v in voices_diff:
    #print(v)
    voices_l.append(data[v].tolist())

voices_l = [item for sublist in voices_l for item in sublist] #to flatten the list

#len(set(voices_l)) #74 unique pitches

In [6]:
window = 32
X = np.empty((0,window), int)
for i in range(0, len(voices_l), window):
    X = np.vstack((X, voices_l[i:i+window]))

In [7]:
len(X)

478

In [8]:
#source: https://jtemporal.com/kmeans-and-elbow-method/
def calculate_wcss(data):
    wcss = []
    for n in range(2, 74):
        kmeans = KMeans(n_clusters=n)
        kmeans.fit(X=data)
        wcss.append(kmeans.inertia_)
   
    return wcss

#source: https://jtemporal.com/kmeans-and-elbow-method/
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = math.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

In [9]:
#wcss = calculate_wcss(X)

In [10]:
#n = optimal_number_of_clusters(wcss)
# 73 ...

In [12]:
#fig, axs = plt.subplots(figsize = (6,6))
#axs.plot(wcss, '-o')

In [13]:
kmeans = KMeans(n_clusters=40) #arbitrarily chosen...
clusters = kmeans.fit_predict(X)

In [14]:
clusters #the clusters / (parts of) themes

array([33, 33, 33, 33, 33, 33, 33,  4, 35, 33,  7,  7, 33, 33, 33, 33, 33,
       33, 33, 33, 35, 33, 33, 34, 33, 33, 33, 35,  0, 36,  9, 35, 33,  7,
       34, 33, 34, 33, 33, 34, 33, 33, 33, 35, 35, 33, 36, 35,  0, 33, 33,
       33, 33, 34, 36, 35, 33,  5, 33, 33, 10, 36,  7, 33,  7, 32, 33, 33,
        1, 33,  8,  7,  1, 10, 36,  7,  7,  0,  1, 33, 33, 33, 33, 33,  6,
       33, 35, 33, 34, 33, 33, 34, 33, 35, 33, 35, 18, 33, 33, 33,  6, 33,
       13,  7, 33, 36, 35, 33, 29, 33, 34, 32,  7, 23, 20, 33, 15, 33,  1,
       33, 33, 33, 33, 33,  4, 35, 33,  7, 34, 34, 33, 33,  7, 33, 35, 33,
       33, 33, 17, 33, 35, 33, 34, 33,  0, 33, 36, 33, 33, 33, 33, 36, 33,
       33, 33,  7, 33, 35, 35, 34, 35, 33, 24, 35, 33, 33, 33, 35, 33, 35,
       33,  0, 33, 39, 34, 33,  0, 36,  7,  7, 33, 32, 33, 36, 35,  0, 39,
       28, 13,  1,  8,  0, 12, 33, 33, 33, 33, 10, 36,  7,  7,  7, 35, 32,
       33, 35, 33, 35, 33, 16, 33, 33, 35, 12, 11, 12,  4, 33, 33, 28, 33,
        1, 20, 33, 33,  9

In [15]:
unique_clusters = np.unique(clusters)
adjacent_clusters = dict()
prev = False
for c in unique_clusters:
    c_dict = dict.fromkeys(unique_clusters,0)
    for val in clusters:
        if prev == c:
            c_dict[val] += 1
            
        prev = val
    
    adjacent_clusters[c] = c_dict          

In [16]:
Transition = []
for i in range(len(adjacent_clusters)):
    row = list(adjacent_clusters[i].values())
    Transition.append(row)
    
for i in range(0,len(Transition)):
    s=sum(Transition[i]) #sum of counts per row (i.e. per pitch)
    for j in range(0,len(Transition[i])):
        Transition[i][j]=Transition[i][j]/s #probability per column

In [17]:
voices_x = []
for v in voices:
    voices_x.append(data[v].tolist())
voices_x = [item for sublist in voices_x for item in sublist] #to flatten the list

unique_windows = []
Z = np.empty((0,window), int)
for i in range(0, len(voices_x), window):
    unique_windows.append(voices_x[i:i+window])
    Z = np.vstack((Z, voices_x[i:i+window]))
unique_windows = [list(x) for x in set(tuple(x) for x in unique_windows)]

In [18]:
Emission=np.zeros([len(unique_clusters),len(unique_windows)])
for i in unique_clusters:
    indices = [x for x, c in enumerate(list(clusters)) if c == i] #get the indices per cluster
    cl_windows = Z[indices] #windows per cluster
    for clw in cl_windows:   
        for cnt, w in enumerate(unique_windows):
            if list(clw) == w: #increas
                Emission[i][cnt] += 1

for i in range(0,len(Emission)):
    Emission[i]=Emission[i]/sum(Emission[i])

In [19]:
obs_pred = []
hid_pred = []
count = 1
offset = 0
for v in voices:
    v_pred = []
    h_pred = []
    start = len(data[v])//32 + offset #to figure out the initial hidden state
    while(count<30):
        
        initial_state = clusters[start]
        hidden_state = int(np.random.choice(unique_clusters,1,p=Transition[initial_state]))
        
        #random_index = np.random.choice(range(len(unique_windows)+1),1)
        observable_state_index = np.random.choice(list(range(len(unique_windows))),p=Emission[hidden_state])
        observable_state = unique_windows[observable_state_index]
        
        v_pred.append(observable_state)
        h_pred.append(hidden_state)
        
        count += 1
        
    obs_pred.append(v_pred)
    hid_pred.append(h_pred)
        
    offset += len(data[v])//32
    
    count = 1

In [20]:
data.drop(columns = ["V1-Diff", "V2-Diff", "V3-Diff", "V4-Diff"], inplace = True)

In [21]:
data_pred = pd.DataFrame({"V1": [item for sublist in obs_pred[0] for item in sublist],
                        "V2": [item for sublist in obs_pred[1] for item in sublist],
                        "V3": [item for sublist in obs_pred[2] for item in sublist],
                        "V4": [item for sublist in obs_pred[3] for item in sublist]})

In [22]:
data_final=data.append(data_pred,ignore_index="True")

In [23]:
data_pred.to_csv("model_themes_pred.csv",sep="\t",header=False,index=False)

In [24]:
data_final.to_csv("model_themes.csv",sep="\t",header=False,index=False)